In [1]:
import pandas as pd
import numpy as np
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from numpy.random import seed
seed(101)
tensorflow.random.set_seed(101)

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.listdir('D:/Анна/универ/ДИПЛОМ/data/ISIC')

['!code',
 'data backup',
 'data backup aug',
 'data backup noaug',
 'ISIC_2019_Test_Input',
 'ISIC_2019_Test_Metadata.csv',
 'ISIC_2019_Training_GroundTruth.csv',
 'ISIC_2019_Training_Input',
 'ISIC_2019_Training_Metadata.csv',
 'test_dir',
 'train.csv',
 'train_dir',
 'val_dir',
 'расшифровка.txt']

In [3]:
base_dir = 'D:/Анна/универ/ДИПЛОМ/data/ISIC'
train_dir = 'D:/Анна/универ/ДИПЛОМ/data/ISIC/train_dir'
val_dir = 'D:/Анна/универ/ДИПЛОМ/data/ISIC/val_dir'
test_dir='D:/Анна/универ/ДИПЛОМ/data/ISIC/test_dir'
os.listdir('D:/Анна/универ/ДИПЛОМ/data/ISIC/train_dir')

['ak', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'unk', 'vasc']

In [4]:
print(len(os.listdir(train_dir+'/nv')))
print(len(os.listdir(train_dir+'/mel')))
print(len(os.listdir(train_dir+'/bkl')))
print(len(os.listdir(train_dir+'/bcc')))
print(len(os.listdir(train_dir+'/ak')))
print(len(os.listdir(train_dir+'/vasc')))
print(len(os.listdir(train_dir+'/scc')))
print(len(os.listdir(train_dir+'/df')))
print(len(os.listdir(train_dir+'/unk')))

1545
542
315
398
104
31
75
29
0


In [5]:
print(len(os.listdir(val_dir+'/nv')))
print(len(os.listdir(val_dir+'/mel')))
print(len(os.listdir(val_dir+'/bkl')))
print(len(os.listdir(val_dir+'/bcc')))
print(len(os.listdir(val_dir+'/ak')))
print(len(os.listdir(val_dir+'/vasc')))
print(len(os.listdir(val_dir+'/scc')))
print(len(os.listdir(val_dir+'/df')))

193
68
40
50
13
3
10
3


In [6]:
# note that we are not augmenting class 'nv'
class_list = ['mel','bkl','bcc','ak','vasc','df', 'scc']

for item in class_list:
    
    # We are creating temporary directories here because we delete these directories later
    # create a base dir
    aug_dir = os.path.join(base_dir,'aug_dir')
    os.mkdir(aug_dir)
    # create a dir within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # list all images in that directory
    img_list = os.listdir(train_dir + '/' + img_class)
    

    # Copy images from the class train dir to the img_dir 
    for fname in img_list:
            src = os.path.join(train_dir , img_class, fname)
            dst = os.path.join(img_dir, fname)
            shutil.copyfile(src, dst)


    # point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = train_dir + '/' + img_class

    # Create a data generator
    datagen = ImageDataGenerator(rescale=1./255,
        rotation_range=90,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

    batch_size = 10

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                                    target_size=(224,224),
                                                    batch_size=batch_size)

    # Generate the augmented images and add them to the training folders
    
    num_aug_images_wanted = 1500 # total number of images we want to have in each class
        
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # run the generator and create about 6000 augmented images
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)
        
    # delete temporary directory with the raw image files
    shutil.rmtree(aug_dir)

Found 542 images belonging to 1 classes.
Found 315 images belonging to 1 classes.
Found 398 images belonging to 1 classes.
Found 104 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 29 images belonging to 1 classes.
Found 75 images belonging to 1 classes.


In [7]:
print(len(os.listdir(train_dir+'/nv')))
print(len(os.listdir(train_dir+'/mel')))
print(len(os.listdir(train_dir+'/bkl')))
print(len(os.listdir(train_dir+'/bcc')))
print(len(os.listdir(train_dir+'/ak')))
print(len(os.listdir(train_dir+'/vasc')))
print(len(os.listdir(train_dir+'/scc')))
print(len(os.listdir(train_dir+'/df')))

1545
1494
1490
1504
1432
1177
1420
1460


In [8]:
class_list = ['nv','mel','bkl','bcc','ak','vasc','df', 'scc']
dir_list= [val_dir, test_dir]
for maindir in dir_list: 
    for item in class_list:

        # We are creating temporary directories here because we delete these directories later
        # create a base dir
        aug_dir = os.path.join(base_dir,'aug_dir')
        os.mkdir(aug_dir)
        # create a dir within the base dir to store images of the same class
        img_dir = os.path.join(aug_dir, 'img_dir')
        os.mkdir(img_dir)

        # Choose a class
        img_class = item

        # list all images in that directory
        img_list = os.listdir(val_dir + '/' + img_class)


        # Copy images from the class train dir to the img_dir 
        for fname in img_list:
                src = os.path.join(val_dir , img_class, fname)
                dst = os.path.join(img_dir, fname)
                shutil.copyfile(src, dst)


        # point to a dir containing the images and not to the images themselves
        path = aug_dir
        save_path = val_dir + '/' + img_class

        # Create a data generator
        datagen = ImageDataGenerator(rescale=1./255)

        batch_size = 50

        aug_datagen = datagen.flow_from_directory(path,
                                               save_to_dir=save_path,
                                               save_format='jpg',
                                                        target_size=(224,224),
                                                        batch_size=batch_size)

        # delete temporary directory with the raw image files
        shutil.rmtree(aug_dir)

Found 193 images belonging to 1 classes.
Found 68 images belonging to 1 classes.
Found 40 images belonging to 1 classes.
Found 50 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 193 images belonging to 1 classes.
Found 68 images belonging to 1 classes.
Found 40 images belonging to 1 classes.
Found 50 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Found 10 images belonging to 1 classes.


In [9]:
print(len(os.listdir(val_dir+'/nv')))
print(len(os.listdir(val_dir+'/mel')))
print(len(os.listdir(val_dir+'/bkl')))
print(len(os.listdir(val_dir+'/bcc')))
print(len(os.listdir(val_dir+'/ak')))
print(len(os.listdir(val_dir+'/vasc')))
print(len(os.listdir(val_dir+'/scc')))
print(len(os.listdir(val_dir+'/df')))

193
68
40
50
13
3
10
3


In [10]:
print(len(os.listdir(test_dir+'/nv')))
print(len(os.listdir(test_dir+'/mel')))
print(len(os.listdir(test_dir+'/bkl')))
print(len(os.listdir(test_dir+'/bcc')))
print(len(os.listdir(test_dir+'/ak')))
print(len(os.listdir(test_dir+'/vasc')))
print(len(os.listdir(test_dir+'/scc')))
print(len(os.listdir(test_dir+'/df')))

193
68
39
50
13
4
9
4


In [11]:
# note that we are not augmenting class 'nv'
class_list = ['nv']
dir_list =[train_dir]
for maindir in dir_list:
    for item in class_list:

        # We are creating temporary directories here because we delete these directories later
        # create a base dir
        aug_dir = os.path.join(base_dir,'aug_dir')
        os.mkdir(aug_dir)
        # create a dir within the base dir to store images of the same class
        img_dir = os.path.join(aug_dir, 'img_dir')
        os.mkdir(img_dir)

        # Choose a class
        img_class = item

        # list all images in that directory
        img_list = os.listdir(maindir + '/' + img_class)


        # Copy images from the class train dir to the img_dir 
        for fname in img_list:
                src = os.path.join(maindir , img_class, fname)
                dst = os.path.join(img_dir, fname)
                shutil.copyfile(src, dst)


        # point to a dir containing the images and not to the images themselves
        path = aug_dir
        save_path = maindir + '/' + img_class

        # Create a data generator
        datagen = ImageDataGenerator(rescale=1./255)

        batch_size = 50

        aug_datagen = datagen.flow_from_directory(path,
                                               save_to_dir=save_path,
                                               save_format='jpg',
                                                        target_size=(224,224),
                                                        batch_size=batch_size)

        # delete temporary directory with the raw image files
        shutil.rmtree(aug_dir)

Found 1545 images belonging to 1 classes.
